In [ ]:
import pandas as pd
import numpy as n
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
df_train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')
Y_test = pd.read_csv('gender_submission.csv')

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
X_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
Y_test.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [8]:
Y_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [9]:
X_test_passenger_id = X_test[['PassengerId']]
df_train = df_train.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
X_test = X_test.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
Y_test = Y_test.drop(['PassengerId'],axis=1)

In [10]:
df_train = pd.get_dummies(df_train,columns=['Sex','Embarked'],drop_first=True)
X_test = pd.get_dummies(X_test,columns=['Sex','Embarked'],drop_first=True)

In [11]:
Y_train = df_train['Survived']
X_train = df_train.drop(['Survived'],axis=1)

In [12]:
Q1 = X_train['Age'].quantile(0.25)
Q3 = X_train['Age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = X_train[(X_train['Age'] < lower_bound) | (X_train['Age'] > upper_bound)]
print(outliers)

     Pclass   Age  SibSp  Parch     Fare  Sex_male  Embarked_Q  Embarked_S
33        2  66.0      0      0  10.5000      True       False        True
54        1  65.0      0      1  61.9792      True       False       False
96        1  71.0      0      0  34.6542      True       False       False
116       3  70.5      0      0   7.7500      True        True       False
280       3  65.0      0      0   7.7500      True        True       False
456       1  65.0      0      0  26.5500      True       False        True
493       1  71.0      0      0  49.5042      True       False       False
630       1  80.0      0      0  30.0000      True       False        True
672       2  70.0      0      0  10.5000      True       False        True
745       1  70.0      1      1  71.0000      True       False        True
851       3  74.0      0      0   7.7750      True       False        True


In [13]:
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].mean())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].mean())

In [14]:
#Decision Tree Classifier
dtClass = DecisionTreeClassifier()
model = dtClass.fit(X_train,Y_train)
model.score(X_test,Y_test)

0.7679425837320574

In [15]:
#Random Forest Classifier
rfClass= RandomForestClassifier()
model = rfClass.fit(X_train,Y_train)
model.score(X_test,Y_test)

0.7894736842105263

In [16]:
#XGBoost Classifier
xgbClass = XGBClassifier()
model = xgbClass.fit(X_train,Y_train)
model.score(X_test,Y_test)

0.8205741626794258

In [17]:
#Gradient Boosting Classifier
gbClass= GradientBoostingClassifier()
model = gbClass.fit(X_train,Y_train)
model.score(X_test,Y_test)

0.8588516746411483

In [18]:
predictions = model.predict(X_test)

In [19]:
output = pd.DataFrame({'PassengerId': X_test_passenger_id.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)